<a href="https://colab.research.google.com/github/barsamnia/knauf_analysis/blob/feature1/pro_knauf_type_Box_14_08_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import os

class MaterialCalculatorDecor:
    def __init__(self):
        self.tool = 0
        self.kaf = 0
        self.pishani = 0
        self.labor_price = 0
        self.material_info = {
            "پنل": {"unit": "برگ", "price_unit": "متر مربع", "unit_size": 2.88, "packaging_rate": 0.055},
            "اف47": {"unit": "شاخه", "price_unit": "متر", "unit_size": 4, "packaging_rate": 0.035},
            "یو36": {"unit": "شاخه", "price_unit": "متر", "unit_size": 4, "packaging_rate": 0.035},
            "نبشی": {"unit": "شاخه", "price_unit": "متر", "unit_size": 4, "packaging_rate": 0.035},
            "پیچ پنل": {"unit": "بسته", "price_unit": "عدد", "unit_size": 1000, "packaging_rate": 0.0},
            "پیچ سازه": {"unit": "بسته", "price_unit": "عدد", "unit_size": 1000, "packaging_rate": 0.0}
        }
        self.unit_prices = {}
        self.quantities = {}
        self.total_prices = {}
        self.packaging_prices = {}
        self.input_history = []

    def get_int_input(self, prompt, allow_back=True):
        while True:
            user_input = input(prompt).strip()
            if user_input.lower() in ['exit', 'ق']:
                print("ورود داده‌ها لغو شد.")
                return None, False
            if allow_back and user_input.lower() in ['back', 'ب']:
                return None, True
            try:
                value = int(user_input)
                if value < 0:
                    print("❌ عدد وارد شده نمی‌تواند منفی باشد.")
                    continue
                return value, False
            except ValueError:
                print("❌ لطفاً یک عدد صحیح وارد کنید یا برای بازگشت 'ب' یا 'back' را وارد کنید.")

    def get_float_input(self, prompt, allow_back=True):
        while True:
            user_input = input(prompt).strip()
            if user_input.lower() in ['exit', 'ق']:
                print("ورود داده‌ها لغو شد.")
                return None, False
            if allow_back and user_input.lower() in ['back', 'ب']:
                return None, True
            try:
                value = float(user_input)
                if value < 0:
                    print("❌ عدد وارد شده نمی‌تواند منفی باشد.")
                    continue
                return value, False
            except ValueError:
                print("❌ لطفاً یک عدد اعشاری یا صحیح وارد کنید یا برای بازگشت 'ب' یا 'back' را وارد کنید.")

    def show_back_menu(self, prompts, inputs):
        print("\n↩️ مراحل قبلی:")
        for i, (prompt, _) in enumerate(prompts[:len(inputs)]):
            print(f"{i + 1}. {prompt.strip(': ')}: {inputs[i]}")
        while True:
            choice = input("شماره مرحله‌ای که می‌خواهید اصلاح کنید (یا 'لغو' برای ادامه): ").strip()
            if choice.lower() in ['لغو', 'cancel']:
                return None
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(inputs):
                    return choice_idx
                print("❌ شماره مرحله نامعتبر است.")
            except ValueError:
                print("❌ لطفاً یک شماره معتبر وارد کنید یا 'لغو' را وارد کنید.")

    def get_inputs(self):
        print("\n🛠 محاسبه مصالح برای باکس (دکور)\n")
        prompts = [
            ("طول پروژه (متر): ", False, self.get_float_input),
            ("عرض کف باکس (سانتی‌متر): ", False, self.get_int_input),
            ("ارتفاع پیشانی باکس (سانتی‌متر): ", False, self.get_int_input),
            ("دستمزد به ازای هر متر طول: ", True, self.get_int_input)
        ]
        prompts.extend([
            (f"💵 قیمت هر {info['price_unit']} از {name} (خروجی به صورت {info['unit']} خواهد بود): ", True, self.get_int_input)
            for name, info in self.material_info.items()
        ])

        inputs = []
        current_step = 0

        while current_step < len(prompts):
            prompt, allow_back, input_func = prompts[current_step]
            print(f"\nمرحله {current_step + 1} از {len(prompts)}")
            value, go_back = input_func(prompt, allow_back)

            if value is None and go_back:
                if not inputs:
                    print("❌ شما در اولین مرحله هستید و نمی‌توانید به عقب برگردید.")
                    continue
                back_step = self.show_back_menu(prompts, inputs)
                if back_step is not None:
                    current_step = back_step
                    inputs = inputs[:current_step]
                    print(f"↩️ بازگشت به مرحله: {prompts[current_step][0]}")
                continue

            if value is None:
                return False

            if current_step == 0 and value == 0:
                print("❌ طول پروژه نمی‌تواند صفر باشد.")
                continue
            if current_step == 1 and value == 0:
                print("❌ عرض کف باکس نمی‌تواند صفر باشد.")
                continue
            if current_step == 2 and value == 0:
                print("❌ ارتفاع پیشانی باکس نمی‌تواند صفر باشد.")
                continue
            if current_step > 3 and value == 0:
                print(f"❌ قیمت {list(self.material_info.keys())[current_step-4]} نمی‌تواند صفر باشد.")
                continue

            if current_step < len(inputs):
                inputs[current_step] = value
            else:
                inputs.append(value)
            print(f"ورودی ثبت‌شده: {prompts[current_step][0].strip(': ')} = {value}")
            current_step += 1

        self.tool = inputs[0]
        self.kaf = inputs[1]
        self.pishani = inputs[2]
        self.labor_price = inputs[3]
        for i, (name, _) in enumerate(self.material_info.items()):
            self.unit_prices[name] = inputs[i + 4]

        self.input_history = inputs
        return True

    def calculate_quantities(self):
        t = self.tool
        kaf = self.kaf / 100  # تبدیل به متر
        pishani = self.pishani / 100  # تبدیل به متر
        self.quantities = {
            "پنل": math.ceil((t * (kaf + pishani + 0.5 * pishani)) / 3),
            "اف47": math.ceil((t * (3 * (kaf - 0.1))) / 4),
            "یو36": math.ceil((t * ((2 * pishani) + 1)) / 4),
            "نبشی": math.ceil((t * 3) / 4),
            "پیچ پنل": math.ceil((t * 65) / 1000),
            "پیچ سازه": math.ceil((t * 6) / 1000)
        }

    def calculate_prices(self):
        for name, info in self.material_info.items():
            qty = self.quantities[name]
            unit_size = info["unit_size"]
            price_per_base_unit = self.unit_prices[name]
            price_per_unit = price_per_base_unit * unit_size
            total_price = qty * price_per_unit
            self.total_prices[name] = total_price
            packaging_rate = info["packaging_rate"]
            self.packaging_prices[name] = total_price * packaging_rate

    def print_summary(self):
        print("\n📋 خلاصه مصالح مورد نیاز برای باکس (دکور):\n")
        for name, info in self.material_info.items():
            qty = self.quantities[name]
            unit = info["unit"]
            price_unit = info["price_unit"]
            base_price = self.unit_prices[name]
            unit_size = info["unit_size"]
            price_per_unit = base_price * unit_size
            total_price = self.total_prices[name]
            packaging_price = self.packaging_prices[name]

            print(f"{name}: {qty} {unit} | قیمت پایه: {base_price} تومان/{price_unit} | قیمت هر {unit}: {price_per_unit:.0f} تومان | مجموع: {total_price:.0f} تومان | بسته‌بندی: {packaging_price:.0f} تومان")

        labor_total = math.ceil(self.labor_price * self.tool )  # دستمزد برای یک طرف باکس
        total_material = sum(self.total_prices.values())
        total_packaging = sum(self.packaging_prices.values())
        warehousing = math.ceil(total_material * 0.01)
        total_extras = total_packaging + warehousing
        total_before_tax = total_material + total_extras
        tax = math.ceil(total_before_tax * 0.1)
        total_with_tax = total_before_tax + tax
        total_with_labor = total_with_tax + labor_total
        price_per_meter = math.ceil(total_with_labor / self.tool)

        print(f"\n🧱 دستمزد: {labor_total} تومان برای {self.tool} متر طول")
        print(f"📦 بسته‌بندی: {total_packaging:.0f} تومان")
        print(f"🏠 انبارداری: {warehousing:.0f} تومان")
        print(f"📌 مالیات: {tax} تومان")
        print(f"💰 جمع کل با مالیات و دستمزد: {total_with_labor} تومان")
        print(f"📐 قیمت هر متر طول: {price_per_meter} تومان")

    def export_to_excel(self, filename="خروجی_باکس.xlsx"):
        df = pd.DataFrame(columns=["نام متریال", "تعداد", "واحد", "قیمت پایه", "واحد پایه", "قیمت کل", "هزینه بسته‌بندی"])
        for name, info in self.material_info.items():
            df.loc[len(df.index)] = {
                "نام متریال": name,
                "تعداد": self.quantities[name],
                "واحد": info["unit"],
                "قیمت پایه": self.unit_prices[name],
                "واحد پایه": info["price_unit"],
                "قیمت کل": self.total_prices[name],
                "هزینه بسته‌بندی": self.packaging_prices[name]
            }

        labor_total = math.ceil(self.labor_price * self.tool * 2)
        df.loc[len(df.index)] = {
            "نام متریال": "دستمزد",
            "تعداد": self.tool,
            "واحد": "متر طول",
            "قیمت پایه": self.labor_price * 2,
            "واحد پایه": "متر طول",
            "قیمت کل": labor_total,
            "هزینه بسته‌بندی": 0
        }

        total_material = sum(self.total_prices.values())
        total_packaging = sum(self.packaging_prices.values())
        warehousing = math.ceil(total_material * 0.01)
        total_extras = total_packaging + warehousing
        total_before_tax = total_material + total_extras
        tax = math.ceil(total_before_tax * 0.1)
        total_with_tax = total_before_tax + tax
        total_with_labor = total_with_tax + labor_total
        price_per_meter = math.ceil(total_with_labor / self.tool)

        df.loc[len(df.index)] = {
            "نام متریال": "اضافات",
            "تعداد": "",
            "واحد": "",
            "قیمت پایه": "",
            "واحد پایه": "",
            "قیمت کل": total_extras,
            "هزینه بسته‌بندی": ""
        }
        df.loc[len(df.index)] = {
            "نام متریال": "مالیات",
            "تعداد": "",
            "واحد": "",
            "قیمت پایه": "",
            "واحد پایه": "",
            "قیمت کل": tax,
            "هزینه بسته‌بندی": ""
        }
        df.loc[len(df.index)] = {
            "نام متریال": "جمع کل با مالیات و دستمزد",
            "تعداد": "",
            "واحد": "",
            "قیمت پایه": "",
            "واحد پایه": "",
            "قیمت کل": total_with_labor,
            "هزینه بسته‌بندی": ""
        }
        df.loc[len(df.index)] = {
            "نام متریال": "قیمت هر متر طول",
            "تعداد": "",
            "واحد": "",
            "قیمت پایه": "",
            "واحد پایه": "",
            "قیمت کل": price_per_meter,
            "هزینه بسته‌بندی": ""
        }

        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"\n📁 فایل خروجی با نام «{filename}» ذخیره شد.")

    def get_excel_path(self):
        default_path = "خروجی_باکس.xlsx"
        user_input = input(f"لطفاً مسیر ذخیره فایل اکسل را وارد کنید (پیش‌فرض: {default_path}): ").strip()
        if user_input == "":
            return default_path
        if not user_input.endswith(".xlsx"):
            user_input += ".xlsx"
        return user_input

    def run(self):
        if not self.get_inputs():
            print("برنامه به دلیل لغو ورودی‌ها متوقف شد.")
            return
        excel_path = self.get_excel_path()
        self.calculate_quantities()
        self.calculate_prices()
        self.print_summary()
        self.export_to_excel(excel_path)

if __name__ == "__main__":
    calc = MaterialCalculatorDecor()
    calc.run()